In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 
from pyspark.sql import Window
import pyspark.sql.types as T

In [2]:
spark=SparkSession.builder.appName("UDF: User Defined Functions").getOrCreate()

22/10/19 11:43:08 WARN Utils: Your hostname, HP-G62 resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/10/19 11:43:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 11:43:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
22/10/19 11:43:11 WARN Utils: Service 'SparkUI' could not bind on

In [4]:
employee_df=spark.read.csv("employees.csv",inferSchema=True,header=True)
employee_df= employee_df.select(["FIRST_NAME","SALARY","DEPARTMENT_ID"])
employee_df.show()

+----------+------+-------------+
|FIRST_NAME|SALARY|DEPARTMENT_ID|
+----------+------+-------------+
|    Donald|  2600|           50|
|   Douglas|  2600|           50|
|  Jennifer|  4400|           10|
|   Michael| 13000|           20|
|       Pat|  6000|           20|
|     Susan|  6500|           40|
|   Hermann| 10000|           70|
|   Shelley| 12008|          110|
|   William|  8300|          110|
|    Steven| 24000|           90|
|     Neena| 17000|           90|
|       Lex| 17000|           90|
| Alexander|  9000|           60|
|     Bruce|  6000|           60|
|     David|  4800|           60|
|     Valli|  4800|           60|
|     Diana|  4200|           60|
|     Nancy| 12008|          100|
|    Daniel|  9000|          100|
|      John|  8200|          100|
+----------+------+-------------+
only showing top 20 rows



In [14]:
def salary_details(salary):
    basic_salary=salary
    ssf=0.1*basic_salary
    tax=0.01*basic_salary
    net_salary=basic_salary-ssf-tax
    return [ssf,tax,net_salary,]

salary_details(10000)
    

[1000.0, 100.0, 8900.0]

In [15]:
# need to register the function to make it available as a DataFrame function:

from pyspark.sql.functions import udf
salary_details_udf = udf(salary_details)

In [19]:
# Then, we can use it in our DataFrame code:
from pyspark.sql.functions import col
employee_df.select(salary_details_udf(col("SALARY")).alias("SSF, Tax, Net Salary")).show()

+--------------------+
|SSF, Tax, Net Salary|
+--------------------+
|[260.0, 26.0, 231...|
|[260.0, 26.0, 231...|
|[440.0, 44.0, 391...|
|[1300.0, 130.0, 1...|
|[600.0, 60.0, 534...|
|[650.0, 65.0, 578...|
|[1000.0, 100.0, 8...|
|[1200.8, 120.08, ...|
|[830.0, 83.0, 738...|
|[2400.0, 240.0, 2...|
|[1700.0, 170.0, 1...|
|[1700.0, 170.0, 1...|
|[900.0, 90.0, 801...|
|[600.0, 60.0, 534...|
|[480.0, 48.0, 427...|
|[480.0, 48.0, 427...|
|[420.0, 42.0, 373...|
|[1200.8, 120.08, ...|
|[900.0, 90.0, 801...|
|[820.0, 82.0, 729...|
+--------------------+
only showing top 20 rows



In [22]:
employee_df=spark.read.csv("employees.csv",inferSchema=True,header=True)
employee_df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [67]:
def email(emal):
    return emal.lower()+"@hotmail.com"
def phone(phon):
    phon=str(phon).split(".")
    return "+977-"+phon[0]+"-"+ phon[1]+"-"+phon[2]

def name(flname):
    naame=flname.title()
    return naame

In [68]:
# need to register the function to make it available as a DataFrame function:

from pyspark.sql.functions import udf
email_udf = udf(email)
phone_udf = udf(phone)
name_udf=udf(name)

In [69]:
# Then, we can use it in our DataFrame code:
from pyspark.sql.functions import col
employee_df.select(email_udf(col("EMAIL")).alias("E mail")).show()

+--------------------+
|              E mail|
+--------------------+
|doconnel@hotmail.com|
|  dgrant@hotmail.com|
| jwhalen@hotmail.com|
|mhartste@hotmail.com|
|    pfay@hotmail.com|
| smavris@hotmail.com|
|   hbaer@hotmail.com|
|shiggins@hotmail.com|
|  wgietz@hotmail.com|
|   sking@hotmail.com|
|nkochhar@hotmail.com|
| ldehaan@hotmail.com|
| ahunold@hotmail.com|
|  bernst@hotmail.com|
| daustin@hotmail.com|
|vpatabal@hotmail.com|
|dlorentz@hotmail.com|
|ngreenbe@hotmail.com|
| dfaviet@hotmail.com|
|   jchen@hotmail.com|
+--------------------+
only showing top 20 rows



In [70]:
employee_df.select(phone_udf(col("PHONE_NUMBER")).alias("Phone NUmber")).show()

+-----------------+
|     Phone NUmber|
+-----------------+
|+977-650-507-9833|
|+977-650-507-9844|
|+977-515-123-4444|
|+977-515-123-5555|
|+977-603-123-6666|
|+977-515-123-7777|
|+977-515-123-8888|
|+977-515-123-8080|
|+977-515-123-8181|
|+977-515-123-4567|
|+977-515-123-4568|
|+977-515-123-4569|
|+977-590-423-4567|
|+977-590-423-4568|
|+977-590-423-4569|
|+977-590-423-4560|
|+977-590-423-5567|
|+977-515-124-4569|
|+977-515-124-4169|
|+977-515-124-4269|
+-----------------+
only showing top 20 rows



In [102]:
fname=employee_df.select(name_udf(col("FIRST_NAME")).alias("First Name"))


In [103]:
lname=employee_df.select(name_udf(col("LAST_NAME")).alias("Last Name"))


In [104]:
fname

DataFrame[First Name: string]

In [106]:

type(lname)

pyspark.sql.dataframe.DataFrame

In [112]:
fullname=fname.join(lname)
fullname.show()

+----------+---------+
|First Name|Last Name|
+----------+---------+
|    Donald| Oconnell|
|    Donald|    Grant|
|    Donald|   Whalen|
|    Donald|Hartstein|
|    Donald|      Fay|
|    Donald|   Mavris|
|    Donald|     Baer|
|    Donald|  Higgins|
|    Donald|    Gietz|
|    Donald|     King|
|    Donald|  Kochhar|
|    Donald|  De Haan|
|    Donald|   Hunold|
|    Donald|    Ernst|
|    Donald|   Austin|
|    Donald|Pataballa|
|    Donald|  Lorentz|
|    Donald|Greenberg|
|    Donald|   Faviet|
|    Donald|     Chen|
+----------+---------+
only showing top 20 rows

